# Improve embedding with HuggingFace pre-trained models

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

## 1. Imports

In [1]:
#classic
import pandas as pd
import numpy as np
from datetime import datetime
import pickle


#visualization
import matplotlib.pyplot as plt

#models
from transformers import AutoTokenizer, TFAutoModel

#import tensorflow as tf

/tmp/ipykernel_44757/4234706728.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/rossnorman11/.pyenv/versions/3.10.6/envs/movie_recommendation_GPT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Using pre-trained tiny-bert

### 1. Text embedding with bert

In [2]:
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name, from_pt = True)

2024-02-13 14:21:54.792481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 14:21:54.792598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 14:21:54.793963: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-13 14:21:54.805776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-13 14:21:57.035857: W tensorflow/compiler/tf2

In [3]:
filename = 'bert.pkl'
pickle.dump(model, open(filename, 'wb'))

/home/rossnorman11/.pyenv/versions/3.10.6/envs/movie_recommendation_GPT/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
2024-02-13 14:24:07.484301: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15627264 exceeds 10% of free system memory.


### 1.1. Embed the (original) plots

In [4]:
# import data
df = pd.read_pickle('/content/drive/MyDrive/shared_data/raw_data_shared/data_movie.pkl')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/shared_data/raw_data_shared/data_movie.pkl'

In [ ]:
plot = df['plot_synopsis'].tolist()

# Tokenize the text data
token_tensor = tokenizer(plot, padding='max_length', max_length= 500, truncation=True, return_tensors="tf")

# Create input tensors
input_tensor = token_tensor['input_ids']

# Generate embeddings
prediction = model.predict(input_tensor)

194/194 [==============================] - 262s 1s/step


2024-02-08 14:28:07.459148: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1588224000 exceeds 10% of free system memory.


In [ ]:
prediction

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=array([[[-0.53041697,  0.07894692, -4.1747184 , ...,  0.1208056 ,
         -0.26163447,  1.4187179 ],
        [-1.2223986 ,  0.2652639 ,  0.31511763, ..., -2.149884  ,
         -0.3219931 ,  1.6733292 ],
        [-0.84478337,  0.3376569 , -0.7529081 , ..., -2.9411268 ,
          0.18474263,  0.29692876],
        ...,
        [-0.64974487,  0.19274138, -1.119652  , ..., -0.16245715,
         -1.546545  ,  1.5404472 ],
        [-0.05792686, -0.09606716, -0.3784339 , ..., -1.3717613 ,
          0.20261516,  0.20651756],
        [-0.73357034,  0.5554948 ,  0.4314581 , ..., -1.3273001 ,
         -0.7229799 ,  0.76031506]],

       [[-1.9052038 , -0.8651169 , -4.592863  , ..., -0.17127931,
         -1.2534415 ,  1.5046825 ],
        [-1.3443586 , -0.42073253, -0.6326513 , ..., -3.1596131 ,
         -0.0952716 ,  0.46136558],
        [-1.5785536 , -0.05970723, -0.8478688 , ..., -1.8083404 ,
         -0.7600982 ,  1.138109  ],
  

In [ ]:
# Process the embeddings as np
plot_embeddings = prediction.last_hidden_state[:, 0, :]

In [ ]:
np.save('/content/drive/MyDrive/shared_data/processed_data_shared/embedding_plot.npy', plot_embeddings)

### 1.2. Embed the (synthetic) summaries

In [ ]:
# import data
df = pd.read_csv('/content/drive/MyDrive/shared_data/raw_data_shared/movie_with_summary.csv')
df.head()

In [ ]:
summaries = df['geny_summary'].tolist()

# Tokenize the text data
token_tensor = tokenizer(summaries, padding='max_length', max_length= 500, truncation=True, return_tensors="tf")

# Create input tensors
input_tensor = token_tensor['input_ids']

# Generate embeddings
prediction = model.predict(input_tensor)

In [ ]:
# Process the embeddings as np
summary_embeddings = prediction.last_hidden_state[:, 0, :]

In [ ]:
np.save('/content/drive/MyDrive/shared_data/processed_data_shared/embedding_summary.npy', summary_embeddings)

**Movie similarity with cosinus similarity**

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

# # Calculate cosine similarity between embeddings
# similarity_matrix = cosine_similarity(embeddings)

# # Example: Get top 10 most similar movies for each movie
# num_movies = similarity_matrix.shape[0]
# top_n = 1  # Number of similar movies to retrieve

# for i in range(num_movies):
#     # Sort similarity scores for movie i
#     sim_scores = sorted(enumerate(similarity_matrix[i]), key=lambda x: x[1], reverse=True)
#     # Exclude movie i itself
#     sim_scores = sim_scores[1:]
#     # Get indices of top similar movies
#     top_indices = [idx for idx, _ in sim_scores[:top_n]]
#     # Print movie titles of top similar movies
#     print(f"Top {top_n} similar movies for '{movie_titles[i]}':")
#     for idx in top_indices:
#         print(f"  - {movie_titles[idx]}")
#     print()

**Movie recommendation with cosine similarity**

In [ ]:
# # Calculate Movie Similarity

# similarity_matrix = cosine_similarity(embeddings)

# # Select User Preferences
# user_input = input("Enter a movie title you like: ")

# # Retrieve Similar Movies
# movie_idx = movie_titles.index(user_input)
# sim_scores = list(enumerate(similarity_matrix[movie_idx]))
# sim_scores_sorted = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# # Filter Recommendations (optional)
# top_n = 5  # Number of recommendations to display
# recommended_movies = [movie_titles[idx] for idx, _ in sim_scores_sorted[1:top_n+1]]  # Exclude the input movie itself

# # Display Recommendations
# print(f"Recommended movies based on '{user_input}':")
# for movie in recommended_movies:
#     print(movie)